In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import sklearn
from sklearn.preprocessing import scale

In [50]:
id_raw_df = pd.read_csv('train_identity.csv')
id_trans_df = pd.read_csv('train_transaction.csv')

In [51]:
id_raw_df.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [52]:
id_trans_df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
train_complete_df = pd.merge(id_trans_df, id_raw_df, on='TransactionID', how='left')

In [54]:
from sklearn.model_selection import train_test_split

In [55]:
columns_def=pd.DataFrame({"na_count":train_complete_df.isnull().sum().sort_values(ascending=False)})

In [56]:
#Create column of percent columns, #higher is worst
columns_def["per"]=columns_def["na_count"]/len(train_complete_df)

In [57]:
limit=0.15 #only columns w/ less 15% will remain

In [58]:
#Select final columns
columns_final=columns_def[columns_def["per"]<limit].index 

train_complete_df=train_complete_df[train_complete_df.columns.intersection(columns_final)]

In [59]:
card6_dummy_df=pd.get_dummies(train_complete_df["card6"])

In [60]:
train_complete_df=pd.concat([train_complete_df,card6_dummy_df],axis=1)

In [61]:
train_complete_df=train_complete_df._get_numeric_data()
train_complete_df=train_complete_df.dropna()

In [62]:
x_complete_df=train_complete_df.drop(["isFraud"],axis=1)
y_complete_df=train_complete_df["isFraud"]

In [63]:
X_train,X_test,y_train,y_test=train_test_split(x_full_df,y_full_df,test_size=0.50)

In [64]:
X_test=X_test.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)

In [65]:
## Modeling

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [66]:
lr = LogisticRegression(solver='lbfgs')
lr.fit(X_test, y_test)

random.seed(100)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [67]:
lr.score(X_test, y_test)

0.979132844619036

In [68]:
y_pred=lr.predict(X_test)

In [69]:
pd.crosstab(y_test, y_pred, rownames=["True"], colnames=["Predicted"], margins=True)

Predicted,0,1,All
True,,,
0,214090,60,214150
1,4503,16,4519
All,218593,76,218669


In [70]:
results_df=pd.DataFrame({"true":y_test, "predicted":y_pred})

In [71]:
results_df[(results_df.true==1)&(results_df.predicted==1)]

,true,predicted
3054,1,1
21308,1,1
24542,1,1
29269,1,1
31478,1,1
45092,1,1
83095,1,1
111789,1,1
112179,1,1
114384,1,1


In [77]:
lr.predict(X_test.iloc[[3054]])

array([1])

In [73]:
X_test.iloc[[107841]].to_json(orient='records')

'[{"TransactionID":3077358,"TransactionDT":1887299,"TransactionAmt":77.0,"card1":16075,"card2":514.0,"card3":150.0,"card5":102.0,"addr1":325.0,"addr2":87.0,"C1":102.0,"C2":96.0,"C3":0.0,"C4":0.0,"C5":96.0,"C6":57.0,"C7":0.0,"C8":0.0,"C9":50.0,"C10":0.0,"C11":80.0,"C12":0.0,"C13":483.0,"C14":70.0,"D1":499.0,"D10":500.0,"V12":0.0,"V13":0.0,"V14":1.0,"V15":0.0,"V16":0.0,"V17":0.0,"V18":0.0,"V19":1.0,"V20":1.0,"V21":0.0,"V22":0.0,"V23":1.0,"V24":1.0,"V25":1.0,"V26":1.0,"V27":0.0,"V28":0.0,"V29":0.0,"V30":0.0,"V31":0.0,"V32":0.0,"V33":0.0,"V34":0.0,"V53":0.0,"V54":0.0,"V55":1.0,"V56":1.0,"V57":0.0,"V58":0.0,"V59":0.0,"V60":0.0,"V61":1.0,"V62":1.0,"V63":0.0,"V64":0.0,"V65":1.0,"V66":1.0,"V67":1.0,"V68":0.0,"V69":0.0,"V70":0.0,"V71":0.0,"V72":0.0,"V73":0.0,"V74":0.0,"V95":0.0,"V96":7.0,"V97":3.0,"V98":0.0,"V99":7.0,"V100":3.0,"V101":0.0,"V102":0.0,"V103":0.0,"V104":0.0,"V105":0.0,"V106":0.0,"V107":1.0,"V108":1.0,"V109":1.0,"V110":1.0,"V111":1.0,"V112":1.0,"V113":1.0,"V114":1.0,"V115":1.0,"V11

In [74]:
import pickle
from sklearn.externals import joblib

In [75]:
joblib.dump(lr, 'model.pkl') 
knn_from_joblib = joblib.load('model.pkl')  
knn_from_joblib.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [76]:
knn_from_joblib.predict_proba(X_test)[0][1]

0.020027163966504495